In [ ]:
## このプログラムは「Digit Recognizer」
## において作成したノートブックにおいて動作します

import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

# train.csvを読み込んでpandasのDataFrameに格納。
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
# trainから画像データを抽出してDataFrameオブジェクトに格納。
train_x = train.drop(['label'], axis=1)
# trainから正解ラベルを抽出してSeriesオブジェクトに格納。
train_y = train['label'] 
# test.csvを読み込んでpandasのDataFrameに格納。
test_x = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

# trainのデータを学習データとテストデータに分ける。
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# 画像のピクセル値を255.0で割って0～1.0の範囲にしてnumpy.arrayに変換。
tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)

# 画像データを2階テンソルを
# (高さ = 28px, 幅 = 28px , チャンネル = 1)の
# 3階テンソルに変換。
# グレースケールのためチャンネルは1。
tr_x = tr_x.reshape(-1,28,28,1)
va_x = va_x.reshape(-1,28,28,1)

# 正解ラベルをOne-Hot表現に変換。
tr_y = to_categorical(tr_y, 10) # numpy.ndarrayオブジェクト
va_y = to_categorical(va_y, 10) # numpy.ndarrayオブジェクト

# x_train、y_train、x_testの形状を出力。
print(tr_x.shape)
print(tr_y.shape)
print(va_x.shape)
print(va_y.shape)

In [ ]:
# 畳み込みニューラルネットワーク

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense, Flatten

model = Sequential()                 # Sequentialオブジェクトの生成

# 第1層
model.add(
    Conv2D(filters=32,               # フィルターの数
           kernel_size=(5, 5),       # 5×5のフィルターを使用
           padding='same',           # ゼロパディングを行う
           input_shape=(28, 28, 1),  # 入力データの形状         
           activation='relu'         # 活性化関数はReLU
           ))

# Flatten層
model.add(Flatten())

# 出力層
model.add(Dense(10,                  # 出力層のニューロン数は10
                activation='softmax' # 活性化関数はsoftmax
               ))
    
# オブジェクトのコンパイル
model.compile(
    loss='categorical_crossentropy', # 損失の基準は交差エントロピー誤差
    optimizer='rmsprop',             # オプティマイザーはRMSprop
    metrics=['accuracy'])            # 学習評価として正解率を指定

# モデルの構造を出力。
model.summary()

In [ ]:
# 学習を行って結果を出力
history = model.fit(
    tr_x,             # 訓練データ
    tr_y,             # 正解ラベル
    epochs=30,        # 学習を繰り返す回数
    batch_size=100,   # ミニバッチの数
    verbose=1,        # 学習の進捗状況を出力する
    validation_data=(
    va_x, va_y        # 検証用データの指定
    ))

In [ ]:
# 損失と正解率（精度）の推移をグラフにする

%matplotlib inline
import matplotlib.pyplot as plt

# プロット図のサイズを設定
plt.ﬁgure(ﬁgsize=(15, 6))
# プロット図を縮小して図の間のスペースを空ける
plt.subplots_adjust(wspace=0.2)

# 1×2のグリッドの左(1,2,1)の領域にプロット
plt.subplot(1, 2, 1)
# 訓練データの損失(誤り率)をプロット
plt.plot(history.history['loss'],
         label='training',
         color='black')
# テストデータの損失(誤り率)をプロット
plt.plot(history.history['val_loss'],
         label='test',
         color='red')
plt.ylim(0, 1)       # y軸の範囲
plt.legend()         # 凡例を表示
plt.grid()           # グリッド表示
plt.xlabel('epoch')  # x軸ラベル
plt.ylabel('loss')   # y軸ラベル

# 1×2のグリッドの右(1,2,21)の領域にプロット
plt.subplot(1, 2, 2)
# 訓練データの正解率をプロット
plt.plot(history.history['accuracy'],
         label='training',
         color='black')
# テストデータの正解率をプロット
plt.plot(history.history['val_accuracy'],
         label='test',
         color='red')
plt.ylim(0.5, 1)     # y軸の範囲
plt.legend()         # 凡例を表示
plt.grid()           # グリッド表示
plt.xlabel('epoch')  # x軸ラベル
plt.ylabel('acc')    # y軸ラベル
plt.show()